> Referenced from GFG and uses VGG16

In [1]:
from keras.models import Model 
from keras.layers import Flatten,Dense 
from keras.applications.vgg16 import VGG16 
import matplotlib.pyplot as plot 
from glob import glob

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
IMAGESHAPE = [224, 224, 3] 
training_data = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
testing_data = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'

In [3]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False) 

58889256/58889256 [==============================] - 0s 0us/step


In [4]:
for each_layer in vgg_model.layers: 
	each_layer.trainable = False

In [5]:
classes = glob('/kaggle/input/chest-xray-pneumonia/chest_xray/train/*') 

In [6]:
flatten_layer = Flatten()(vgg_model.output) 
prediction = Dense(len(classes), activation='softmax')(flatten_layer) 

In [7]:
final_model = Model(inputs=vgg_model.input, outputs=prediction) 
final_model.summary() 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
final_model.compile( 
loss='categorical_crossentropy', 
optimizer='adam', 
metrics=['accuracy'] 
)

In [9]:
from keras.preprocessing.image import ImageDataGenerator 
train_datagen = ImageDataGenerator(rescale = 1./255, 
								shear_range = 0.2, 
								zoom_range = 0.2, 
								horizontal_flip = True) 
testing_datagen = ImageDataGenerator(rescale =1. / 255)

In [11]:
training_set = train_datagen.flow_from_directory('/kaggle/input/chest-xray-pneumonia/chest_xray/train', 
												target_size = (224, 224), 
												batch_size = 4, 
												class_mode = 'categorical') 

Found 5216 images belonging to 2 classes.


In [12]:
test_set = testing_datagen.flow_from_directory('/kaggle/input/chest-xray-pneumonia/chest_xray/test', 
											target_size = (224, 224), 
											batch_size = 4, 
											class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [13]:
fitted_model = final_model.fit_generator( 
training_set, 
validation_data=test_set, 
epochs=5, 
steps_per_epoch=len(training_set), 
validation_steps=len(test_set) 
)

/tmp/ipykernel_32/108269192.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fitted_model = final_model.fit_generator(


Epoch 1/5
1304/1304 [==============================] - 153s 112ms/step - loss: 1.0932 - accuracy: 0.9066 - val_loss: 1.5964 - val_accuracy: 0.9151
Epoch 2/5
1304/1304 [==============================] - 116s 89ms/step - loss: 0.9541 - accuracy: 0.9344 - val_loss: 6.0561 - val_accuracy: 0.7628
Epoch 3/5
1304/1304 [==============================] - 118s 90ms/step - loss: 0.9058 - accuracy: 0.9452 - val_loss: 1.5199 - val_accuracy: 0.9327
Epoch 4/5
1304/1304 [==============================] - 119s 91ms/step - loss: 0.8901 - accuracy: 0.9465 - val_loss: 3.4321 - val_accuracy: 0.8606
Epoch 5/5
1304/1304 [==============================] - 118s 90ms/step - loss: 1.0141 - accuracy: 0.9417 - val_loss: 2.7027 - val_accuracy: 0.9183


In [14]:
final_model.save('pneumonia_model.h5')

In [24]:
from tensorflow.keras.preprocessing import image 
from keras.models import load_model 
from keras.applications.vgg16 import preprocess_input 
import numpy as np 

model=load_model('pneumonia_model.h5') #Loading our model 
img=image.load_img('/kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1440-0001.jpeg',target_size=(224,224)) 
imagee=image.img_to_array(img) #Converting the X-Ray into pixels 
imagee=np.expand_dims(imagee, axis=0) 
img_data=preprocess_input(imagee) 
prediction=model.predict(img_data)

print(prediction)

if prediction[0][0]>prediction[0][1]: #Printing the prediction of model. 
	print('Person is safe.') 
else: 
	print('Person is affected with Pneumonia.') 
print(f'Predictions: {prediction}') 

1/1 [==============================] - 0s 138ms/step
[[0. 1.]]
Person is affected with Pneumonia.
Predictions: [[0. 1.]]


In [25]:
from tensorflow.keras.preprocessing import image 
from keras.models import load_model 
from keras.applications.vgg16 import preprocess_input 
import numpy as np 

model=load_model('pneumonia_model.h5') #Loading our model 
img=image.load_img('/kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg',target_size=(224,224)) 
imagee=image.img_to_array(img) #Converting the X-Ray into pixels 
imagee=np.expand_dims(imagee, axis=0) 
img_data=preprocess_input(imagee) 
prediction=model.predict(img_data) 

print(prediction)

if prediction[0][0]>prediction[0][1]: #Printing the prediction of model. 
	print('Person is safe.') 
else: 
	print('Person is affected with Pneumonia.') 
print(f'Predictions: {prediction}') 

1/1 [==============================] - 0s 136ms/step
[[0. 1.]]
Person is affected with Pneumonia.
Predictions: [[0. 1.]]
